# Retrivers
A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) it. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.

In [ ]:
!pip install python-dotenv chromadb langchain openai

In [1]:
import os
from dotenv import load_dotenv
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyMuPDFLoader

This example showcases question answering over documents.
Question answering over documents consists of four steps:

1. Create an index
2. Create a Retriever from that index
3. Create a question answering chain
4. Ask questions!

In [3]:
loader = PyMuPDFLoader("data/MetaAI - LLM guide with Llama2, fine tuning.pdf")

# One Line Index Creation
To get started as quickly as possible, we can use the `VectorstoreIndexCreator`.

In [4]:
from langchain.indexes import VectorstoreIndexCreator

index = VectorstoreIndexCreator().from_loaders([loader])

Now that the index is created, we can use it to ask questions of the data! Note that under the hood this is actually doing a few steps as well, which we will cover later in this guide.

In [5]:
query = "What is responsible AI and how Llama2 is implementing it?"
index.query(query)

' Responsible AI is a set of considerations to ensure that generative AI technology does not produce undue harm. Llama2 is implementing responsible AI by providing resources and best practices for the responsible development of downstream LLM-powered features, and by taking a commitment to building responsible AI seriously.'

In [6]:
query = "How to determine use case for fine tunning a model?"
index.query_with_sources(query)

{'question': 'How to determine use case for fine tunning a model?',
 'answer': ' To determine a use case for fine tuning a model, developers should identify a specific product use case and assess the risks associated with that use case. They should also apply best practices to ensure safety.\n',
 'sources': 'data/MetaAI - LLM guide with Llama2, fine tuning.pdf'}

What is returned from the `VectorstoreIndexCreator` is `VectorStoreIndexWrapper`, which provides these nice `query` and `query_with_sources` functionality. If we just wanted to access the vectorstore directly, we can also do that.

In [7]:
index.vectorstore

If we then want to access the `VectorstoreRetriever`, we can do that with:

In [8]:
index.vectorstore.as_retriever()

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], metadata=None, vectorstore=<langchain.vectorstores.chroma.Chroma object at 0x7f549dba2310>, search_type='similarity', search_kwargs={})

`VectorstoreIndexCreator` is just a wrapper around all this logic. It is configurable in the text splitter it uses, the embeddings it uses, and the vectorstore it uses. For example, you can configure it as below:

In [9]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

index_creator = VectorstoreIndexCreator(
    vectorstore_cls=Chroma,
    embedding=OpenAIEmbeddings(),
    text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
)

# MultiQueryRetriever
The MultiQueryRetriever automates the process of prompt tuning by using an LLM to generate multiple queries from different perspectives for a given user input query. For each query, it retrieves a set of relevant documents and takes the unique union across all queries to get a larger set of potentially relevant documents. By generating multiple perspectives on the same question, the MultiQueryRetriever might be able to overcome some of the limitations of the distance-based retrieval and get a richer set of results.

In [17]:
# Build a sample vectorDB
from langchain.vectorstores import Chroma
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load blog post about how to convert carpool
loader = WebBaseLoader("https://www.redfin.com/blog/carport-garage-conversion-what-to-consider/")
data = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(data)

# VectorDB
embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=splits, embedding=embedding)

Specify the LLM to use for query generation, and the retriver will do the rest.

In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever

question = "Why people want to convert carports?"

retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(), 
    llm=ChatOpenAI(temperature=0)
)

In [19]:
# Set logging for the queries
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [20]:
# Run query 
unique_docs = retriever_from_llm.get_relevant_documents(query=question)
len(unique_docs)

INFO:langchain.retrievers.multi_query:Generated queries: ["1. What are the reasons behind people's desire to convert carports?", '2. What motivates individuals to convert carports?', '3. What factors drive people to consider converting carports?']


4